In [1]:
from youper import (get_data, ReflectionModel, generate_reflection_from_tensors, 
                    observe_reflection, generate_reflection_from_string)

from transformers import RobertaModel, RobertaTokenizer
import torch

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df = get_data()
df.shape

(2271, 14)

In [4]:
df.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,split,first_an_sent,seems_sounds_sents,reflection
0,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,"If everyone thinks you're worthless, then mayb...",1,2899,train,"If everyone thinks you're worthless, then mayb...",None,"If everyone thinks you're worthless, then mayb..."
1,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Robin Landwehr, DBH, LPCC, NCCMental Health in...",https://counselchat.com/therapists/robin-landw...,"Hello, and thank you for your question and see...",1,3514,train,"Hello, and thank you for your question and see...",None,"Hello, and thank you for your question and see..."
2,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Lee KingI use an integrative approach to treat...,https://counselchat.com/therapists/lee-king,First thing I'd suggest is getting the sleep y...,0,5,train,First thing I'd suggest is getting the sleep y...,None,First thing I'd suggest is getting the sleep y...
3,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,"Shauntai Davis-YearginPersonalized, private on...",https://counselchat.com/therapists/shauntai-da...,Therapy is essential for those that are feelin...,0,31,train,Therapy is essential for those that are feelin...,None,Therapy is essential for those that are feelin...
4,0,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,https://counselchat.com/questions/can-i-change...,depression,Jordan WhiteLicensed Social Worker at Oak Root...,https://counselchat.com/therapists/jordan-white,I first want to let you know that you are not ...,0,620,train,I first want to let you know that you are not ...,None,I first want to let you know that you are not ...


## Train / Valid / Test Split

In [5]:
train_mask = df.split == 'train' 
valid_mask = df.split == 'val'
test_mask = df.split == 'test'
train_mask.sum(), valid_mask.sum(), test_mask.sum()

(1963, 185, 123)

## Pretrained RoBERTA encoder

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta = RobertaModel.from_pretrained('roberta-base')

## Reflection model

In [7]:
addl_state = {
    'df': df,
    'train_mask': train_mask,
    'valid_mask': valid_mask,
    'tokenizer': tokenizer}

model = ReflectionModel(
    roberta=roberta, 
    dec_hidden_sz=512,
    addl_state=addl_state)

In [8]:
model.load_state_dict(torch.load('models/reflections.pt'))

<All keys matched successfully>

## GENERATE REFLECTIONS

In [9]:
start_token_id = tokenizer.convert_tokens_to_ids('<s>')
end_token_id = tokenizer.convert_tokens_to_ids('</s>')
start_token_id, end_token_id

(0, 2)

In [12]:
%%time

def gen_reflection(s):
    pred_toks = generate_reflection_from_string(model, tokenizer, s, start_token_id, end_token_id)
    return tokenizer.decode(pred_toks)

df['gen_reflection'] = df.questionText.apply(gen_reflection)

CPU times: user 2min 5s, sys: 18.1 s, total: 2min 24s
Wall time: 2min 24s


## Observe some generated reflections

In [13]:
def show_example(df):
    sample = df.sample(1)
    print('='*70)

    print('QUESTION:')
    print(sample['questionText'].iloc[0])

    print('\nACTUAL REFLECTION:')
    print(sample['reflection'].iloc[0])

    print('\nGENERATED REFLECTION:')
    print(sample['gen_reflection'].iloc[0])
    print()

### Training set

In [27]:
for _ in range(5): show_example(df[train_mask])

QUESTION:
She's busy because her mom makes her clean all the time and go out places with her family. We don't talk much because of it. Also, we have little fights. We want to work it out but we don't know how.

ACTUAL REFLECTION:
Maybe your girlfriend feels torn in her emotions between loyalty toward her family and toward investing herself in a relationship.There are so many "maybes", that the best way to strengthen your relationship is to ask your girlfriend if she feels any

GENERATED REFLECTION:
 I'm sorry to hear that you are feeling torn and the relationship you are feeling.

QUESTION:
I have been with a guy for 4 years, he's a great guy and we also have a son together. The problem is that I'm in love with a guy that I've been talking to for about 2 years but I've never met him in person. Honestly I'm bored with the relationship I have with the first guy and he makes want to go after the second guy, I don't know how to tell him that. What should I do?

ACTUAL REFLECTION:
Hi there.

### Validation set

In [26]:
for _ in range(5): show_example(df[valid_mask])

QUESTION:
I’m a teenager. My entire family needs family therapy, and more than likely individual therapy. My parents refuse to take action, and I'm tired of it. Is there any way I can get out of this myself?

ACTUAL REFLECTION:
This sounds like a really tough situation.

GENERATED REFLECTION:
 I'm sorry to hear that you are feeling pretty depressed.

QUESTION:
I’m trying to make marriage work after a split. Before our split, he lied a lot and broke every promise to me. I don't think he cheated. Last month, I asked what women work with him, so he told me. Yesterday, I found out about a girl that he said he forgot about. Should I be upset?

ACTUAL REFLECTION:
Are you upset, is the more pertinent question.Everyone has their own tolerance for lies and their unique reasons for being married.

GENERATED REFLECTION:
 I'm sorry to hear about your relationship about your relationship with your relationship with you.

QUESTION:
I empathize so much, even with characters on tv or in video games, t

In [14]:
prompt = "A character in a book I'm reading died"
tokenizer.decode(generate_reflection_from_string(
    model, tokenizer, prompt, start_token_id, end_token_id))

' I think that you are not a good question.'

In [15]:
prompt = """I am lonely. I feel there is nobody I can talk to.
Nobody to hang-out with. I really feel I can't make any friends"""

tokenizer.decode(generate_reflection_from_string(
    model, tokenizer, prompt, start_token_id, end_token_id))

' I am so sorry to hear about your question.'

In [16]:
prompt = """My mother doesn't trust me or believe me in anything I say. 
I have lost trust wth her. And i no longer respect her"""

tokenizer.decode(generate_reflection_from_string(
    model, tokenizer, prompt, start_token_id, end_token_id))

' I am sorry to hear that you are not alone.'

In [17]:
prompt = """Uncertainty about the future. Nothing to look forward too.
Impasse with my house sale"""

tokenizer.decode(generate_reflection_from_string(
    model, tokenizer, prompt, start_token_id, end_token_id))

' I would suggest that you are a good question.'